# D2E2F: ForSea meeting
(2021-04-29)

### 1) Utveckla simuleringsmodell med Machine Learning
### 2) Använda simuleringsmodell för att optimera thrustrar 

### 3) Analysarbete
#### a) Långtidsanalys: dagar/veckor/månader/år
#### b) Förbättring med "Smart Eco-shipping"
#### c) Korttidsanalys: farygsdynamik under resa

### 4) PhD Chalmers paper
### 5) "Förstå" datan
![](figures/I_want_to_understand.png)

In [28]:
#%load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)
plt.style.use('presentation')

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os
sys.path.append('../')
from src.visualization import visualize, animate
sys.path.append('../src/models/pipelines/longterm/scripts/prepdata/trip')
import prepare_dataset, trips, trip_id
sys.path.append('../src/models/pipelines/longterm/scripts/prepdata/clean_statistics')
import clean_statistics


import scipy.integrate
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
experiments_path = '../notebooks/pipelines/longterm/experiments'
experiment = 'tycho_short_parquet'
experiment_path = os.path.join(experiments_path, experiment)
file_path = os.path.join(experiment_path, 'id.parquet')
df = trip_id.load_output_pandas(path=file_path)

In [24]:
df.head()

,latitude,longitude,heading,power_em_thruster_1,power_em_thruster_2,power_em_thruster_3,power_em_thruster_4,power_em_thruster_total,cog,sin_pm1,sin_pm2,sin_pm3,sin_pm4,cos_pm1,cos_pm2,cos_pm3,cos_pm4,power_heeling,sog,trip_no,trip_time,reversing,trip_direction
time,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01 08:31:19+00:00,56.0331,12.61723,77.7,146.0,123.0,148.0,164.0,581.0,89.04,-0.20230,-0.15491,0.01044,-0.01880,-0.98050,-0.99164,-1.0,-0.99954,0.0,0.216067,1.0,0.0,False,0
2020-01-01 08:31:20+00:00,56.0331,12.61723,77.6,164.0,166.0,150.0,162.0,642.0,86.49,-0.10016,-0.09088,0.02536,-0.00851,-0.98849,-0.99707,-1.0,-0.99966,0.0,0.205778,1.0,1.0,False,0
2020-01-01 08:31:21+00:00,56.0331,12.61723,77.5,171.0,177.0,146.0,162.0,656.0,84.19,-0.07849,-0.02191,0.02719,-0.00839,-0.99469,-0.99997,-1.0,-0.99969,0.0,0.246933,1.0,2.0,False,0
2020-01-01 08:31:22+00:00,56.0331,12.61724,77.4,182.0,186.0,152.0,159.0,679.0,81.75,-0.05557,0.00128,0.02710,-0.00848,-0.99591,-0.99997,-1.0,-0.99969,0.0,0.288089,1.0,3.0,False,0
2020-01-01 08:31:23+00:00,56.0331,12.61724,77.3,203.0,205.0,150.0,158.0,716.0,80.01,-0.04800,0.00131,0.02771,-0.00830,-0.99615,-0.99994,-1.0,-0.99954,0.0,0.288089,1.0,4.0,False,0


## Plot trips

In [26]:
visualize.plot_trips(df=df, width=1400, height=800, zoom_start=14, color_key='trip_direction')

## Varför åker man inte raka vägen?

## Trip statistics

In [34]:
experiments_path = '../notebooks/pipelines/longterm/experiments'
experiment2 = '2020-01-01-till-2021-02-28-tycho-brahe'
experiment_path2 = os.path.join(experiments_path, experiment2)

file_path = os.path.join(experiment_path2,'id_statistics_clean.parquet')
df_stat = clean_statistics.load_output_as_pandas_dataframe(path=file_path)

In [33]:
fig = px.scatter(df_stat, x='start_time',y='P', color='trip_direction', width=1500, height=600, 
                color_discrete_sequence=['red','green'])
fig.show()

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['latitude', 'longitude', 'heading', 'power_em_thruster_1', 'power_em_thruster_2', 'power_em_thruster_3', 'power_em_thruster_4', 'power_em_thruster_total', 'cog', 'sin_pm1', 'sin_pm2', 'sin_pm3', 'sin_pm4', 'cos_pm1', 'cos_pm2', 'cos_pm3', 'cos_pm4', 'power_heeling', 'sog', 'trip_no', 'trip_time', 'reversing', 'trip_direction', 'start_time', 'end_time', 'start_index', 'end_index'] but received: P

## Analyzing trips

In [ ]:
fig = px.line(df, x='trip_time', y='sog', color='trip_no', width=1500, height=800)
fig.show()

![](animations/trip_101.gif "101")

## Look at one trip

In [ ]:
trips = df.groupby(by='trip_no')
trip_no = 129
trip = trips.get_group(trip_no)

In [ ]:
visualize.plot_map(df=trip, width=1400, height=800, zoom_start=14, color_key='trip_direction')

In [ ]:
trip[[f'sin_pm{i}' for i in range(1,5)]].isnull().any()

In [ ]:
trip[['cog','heading','sog']].isnull().any()

In [ ]:
animate.widget(trip=trip)

## Is there a more optimal thruster operation?

![](https://memegenerator.net/img/images/15921176.jpg)

## Thruster positions?

Annas definition:

    Thruster 1 – NV
    Thruster 2 – SV
    Thruster 3 – NE
    Thruster 4 - SE


In [ ]:
df_no_reverse = df.groupby('trip_no').filter(lambda x : x.iloc[0]['reversing']==0)
groups = df_no_reverse.groupby('trip_no')
trip_no_reverse =groups.get_group(list(groups.groups.keys())[0])
trip_ = animate.normalize_power(trip=trip_no_reverse)
row = trip_.iloc[500].copy()




fig,ax=plt.subplots()
fig.set_size_inches(6,6)
animate.plot_thrusters(ax=ax, row=row, positions=['NV', 'SV', 'NE', 'SE'])
ax.set_title(f'reversing:{row["reversing"]}, trip_direction: {row["trip_direction"]}');

In [ ]:
animate.widget(trip=trip, positions=['NV', 'SV', 'NE', 'SE'])

## Transverse force for thrusters during one trip

In [ ]:
sin_keys = ['sin_pm%i' % n for n in range(1,5)]
cos_keys = ['cos_pm%i' % n for n in range(1,5)]
power_keys = ['power_em_thruster_%i' % n for n in range(1,5)]
columns = sin_keys + cos_keys + power_keys

In [ ]:
g = sns.PairGrid(trip[sin_keys])
g.map_upper(sns.scatterplot)
#g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False);

## Compare mean power for many trips

In [ ]:
df_mean =  df.groupby('trip_no').mean()

In [ ]:
g = sns.PairGrid(df_mean[power_keys + ['reversing']])
g.map_upper(sns.scatterplot)
#g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False);

In [ ]:
df_no_reverse = df.groupby('trip_no').filter(lambda x : x.iloc[0]['reversing']==1)
groups = df_no_reverse.groupby('trip_no')
trip_no_reverse =groups.get_group(list(groups.groups.keys())[0])
trip_ = animate.normalize_power(trip=trip_no_reverse)
row = trip_.iloc[500].copy()

fig,ax=plt.subplots()
fig.set_size_inches(6,6)
animate.plot_thrusters(ax=ax, row=row)
ax.set_title(f'reversing:{row["reversing"]}, trip_direction: {row["trip_direction"]}');

In [ ]:
df_no_reverse = df.groupby('trip_no').filter(lambda x : x.iloc[0]['reversing']==0)
groups = df_no_reverse.groupby('trip_no')
trip_no_reverse =groups.get_group(list(groups.groups.keys())[0])
trip_ = animate.normalize_power(trip=trip_no_reverse)
row = trip_.iloc[500].copy()

fig,ax=plt.subplots()
fig.set_size_inches(6,6)
animate.plot_thrusters(ax=ax, row=row)
ax.set_title(f'reversing:{row["reversing"]}, trip_direction: {row["trip_direction"]}');

## Fartygsdata?
![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/20190531_Fersea_Tycho_Brahe_Helsingor_0120_%2847981263576%29.jpg/335px-20190531_Fersea_Tycho_Brahe_Helsingor_0120_%2847981263576%29.jpg)
* GA?
* Max effekt på thrustrar?

## Nästa möte...